## Hedging strategies:

- Delta hedging:

    $$\Delta = \frac{\partial C^{BS}}{\partial S} = \mathcal{N}(d_1)$$

    where,
    
    $\Delta =$ delta, $C=$ Call Option, $BS=$  Black-Scholes, $S =$ Underlying and $d_1 =$ first term from the Black-Scholes.

    Underlying asset = NVDA.O

    Maturity Date = 2024-10-25
    
    T = 45
    
    Start date = 2024-09-10

In [107]:
import datetime as dt
import pandas as pd
import numpy as np
from scipy.stats import norm

date = dt.datetime.strptime('2024-10-25', '%Y-%m-%d')
t = 45 - (date - dt.datetime.strptime('2024-09-11', '%Y-%m-%d')).days
date1 = date - dt.timedelta(days=45)
print(date1, t)

data = pd.read_feather('firma1.feather')
# drop unnecessary three rows as our hedging starting day is 2024-09-10
df = data.drop(index=[0,1,2]).reset_index(drop=True)

2024-09-10 00:00:00 1


In [108]:
def calculate_delta(S, K, T, t, r, sigma, eps):
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    return norm.cdf(d1)

$$d_1 = \frac{\ln (S/T)+(r+\sigma^2/2)(T-t)}{\sigma\sqrt{T-t}}$$
$$d_2 = d_1 - \sigma \cdot \sqrt{T-t}$$

In [109]:
# Black-Scholes Call Price
def black_scholes_call(S, K, T, t, r, sigma, eps):
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [110]:
def interval_func(value, interval, amount):
    result = []
    start = value - interval * (amount//2)
    for i in range(amount):
        result.append(round(start,4))
        start += interval
    return result

#interval_func(0.5, 0.001, 1000)

In [111]:
# ATM 2024-09-10
T = 45/365
C_0 = 10
S_0 = 108.1
t = 0
K = 105
# Risk-free rate is us 30-day treasure bond risk-free rate at 2024-09-10
r = 0.0497
eps = np.finfo(float).eps
# sigma = ?

In [112]:
"""
S = Underlying price at the date
K = Strike Price
T = Time to maturity (years)
t = day now
interval = re-hedging interval (days)
C_= Call price

Func finds volatility to the call option and calculates delta
"""

def calc_delta(S, K, T, t, r, C_0, eps=np.finfo(float).eps, tol=1):

    for sigma in interval_func(0.5, 0.001, 1000):
        c = black_scholes_call(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
        if abs(c-C_0) <= tol:
            delta = calculate_delta(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
            print(f"Delta = {delta}, sigma = {sigma} call price from BS = {round(c, 5)}")
            return delta
    print(c)
    return None

#calc_delta(S_0, K, T, t, r, C_0)

In [113]:
black_scholes_call(S=S_0, K=K, T=T, t=t, r=r, sigma=0.53789, eps=eps)

10.000999061418113

In [114]:
# ATM 2024-09-10
T = 45/365
C_0 = 10
S_0 = 108.1
t = 0
K = 105

# Risk-free rate is us 30-day treasure bond risk-free rate at 2024-09-10
r = 0.0497
eps = np.finfo(float).eps
sigma = 0.53789

In [115]:
delta = calculate_delta(S=S_0, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
print(f"Delta = {delta}")

Delta = 0.610619678591889


In [116]:
delta * S_0

66.0079872557832

We have n=45 days to the maturity $t_0=0$. We have one call option with price $C_0 = 10$ on that date 2024-09-10. Then we have another portfolio containing delta amount of underlying asset which is $\Delta_0 \cdot S_0 = 0.6073354 \cdot 108.1 = 65.6530$

In [117]:
"""
 Now we compute how much the value of the OP and RE changes on each day.
 OP value change is calculated as C_1 - C_0 where C_0 is value before C_1. Data is stored to the OP_value array in increasing date order
 as the original data in the firma1.feather


"""

c_0 = 0.0
OP_value = []

for i, row in df.iterrows():
    if i == 3:
        c_0 = float(row['C105'])
        continue

    c_1 = float(row['C105'])
    c_out = c_1 - c_0

    OP_value.append(round(c_out, 4))
    c_0 = float(row['C105'])

In [118]:
"""
 RE value change is calculated as s_1 - s_0 where s_0 is underlying value before s_1. Data is stored to the RE_value array in increasing date order
 as the original data in the firma1.feather
"""

s_0 = 0
RE_value = []


for i, row in df.iterrows():
    if i == 3:
        s_0 = float(row['Underlying'])
        continue

    s_1 = float(row['Underlying'])

    s_out = s_1 - s_0
    RE_value.append(round(s_out, 4))
    s_0 = float(row['Underlying'])

In [119]:
# ATM 2024-09-10
T = 45/365
C_0 = 10
S_0 = 108.1
t = 0
K = 105

# Risk-free rate is us 30-day treasure bond risk-free rate at 2024-09-10
r = 0.0497
eps = np.finfo(float).eps

In [120]:
"""
We want to re-hedge the portfolio at specific intervals and calculate mean square error E = (1 / n - 1) * SUM_i=1->n-1(A^2)
We choose to hedge every second day.
Function calculates both OP and RE portfolio values and their difference A_i as the result.
Re-hedging is done by calculating new delta values.

OP = c_i+1 - c_1
RE = delta_i(s_i+1 - s_i)
A_i = OP + RE
E = (1 / n - 1) * SUM_i=1->n-1(A_i^2)

interval = re-hedging interval (days)
strike = strike price in format ('C{strike_price}) e.g. 'C105'
"""

OP_0 = C_0

# delta * S_0
RE_0 = 0.6073354 * S_0

maturity_date = dt.datetime.strptime('2024-10-25', '%Y-%m-%d')

def hedging(interval, strike, df):
    A_boss = 0
    interval_count = 1
    c_0 = C_0
    c_1 = 0
    s_0 = S_0
    s_1 = 0
    delta = 0.6073354
    t = 0
    n = 0

    for _, row in df.iterrows():
        c_1 = float(row[strike])
        s_1 = float(row['Underlying'])
        t = (45 - (maturity_date - row['Date']).days) / 365
        
        if row.name == 1:
            continue

        OP = c_1 - c_0

        
        RE = delta * (s_1-s_0)
        A = OP + RE
        A_boss += A ** 2
        n += 1
        
        if interval_count % interval == 0:
          print(row['Date'].strftime("%Y-%m-%d"))
          delta = calc_delta(s_1, K, T, t, r, c_1)
          

        c_0 = c_1
        s_0 = s_1
        interval_count += 1

    mse = A_boss/(1-n)
    
    return mse



In [121]:
hedging(interval=2, strike='C105', df=df)

2024-09-12
Delta = 0.803650096624295, sigma = 0.501 call price from BS = 17.33458
2024-09-16
Delta = 0.8068221896405163, sigma = 0.429 call price from BS = 14.6071
2024-09-18
Delta = 0.7706403562193196, sigma = 0.377 call price from BS = 11.33185
2024-09-20
Delta = 0.8733267546305267, sigma = 0.308 call price from BS = 12.65764
2024-09-24
Delta = 0.9700688695631603, sigma = 0.27 call price from BS = 16.75161
2024-09-26
Delta = 1.0, sigma = 0.0 call price from BS = 19.68141
2024-09-30
Delta = 1.0, sigma = 0.0 call price from BS = 17.08141
2024-10-02
Delta = 0.9700358899272781, sigma = 0.274 call price from BS = 14.80412
2024-10-04
Delta = 1.0, sigma = 0.0 call price from BS = 20.56141
2024-10-08
Delta = 1.0, sigma = 0.0 call price from BS = 28.53141
2024-10-10
Delta = 1.0, sigma = 0.0 call price from BS = 30.45141
2024-10-14
Delta = 1.0, sigma = 0.0 call price from BS = 33.71141
2024-10-16
Delta = 1.0, sigma = 0.0 call price from BS = 31.36141
2024-10-18
Delta = 1.0, sigma = 0.0 call pr

-36.86996838610861